# 获取电影评论数据集

In [1]:
import pyprind

In [1]:
import pandas as pd

In [3]:
import os

In [2]:
import numpy as np

生成一个50000次迭代的进度条

In [5]:
pbar = pyprind.ProgBar(50000) 

In [6]:
labels = {'pos':1,'neg':0}

In [7]:
df = pd.DataFrame()

对语料库文件夹进行遍历，将测试集test\训练集train\正例样本集pos\负例样本集neg导入dataFrame
os.listdir(path) 会返回包含路径path下所有文件的列表
os.path.join(path,file) 会返回一个path和file的组合地址(用/连接)

In [9]:
for s in ('test','train'):
    for l in ('pos','neg'):
        path = '/home/fannian/too_data/aclImdb/%s/%s' % (s,l)
        for file in os.listdir(path):
            with open(os.path.join(path,file), 'r') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()#更新进度条

0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:40


In [10]:
df.columns = ['review', 'sentiment']

In [11]:
np.random.seed(0)

np.random.permutation()对数据结构(df)的索引编号进行随机打乱
df.reindex(newindex) 用新的索引编号替换原有索引，若原有索引数据不存在，则为NaN

In [12]:
df = df.reindex(np.random.permutation(df.index))

In [13]:
df.to_csv('/home/fannian/tool_data/outdata/movie_data.csv',index = False)

# 词袋模型

In [49]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fannian/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
import re

In [3]:
import nltk

In [4]:
from nltk.stem.porter import PorterStemmer

In [5]:
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

In [7]:
from sklearn.naive_bayes import MultinomialNB

In [8]:
from nltk.corpus import stopwords

In [9]:
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score

In [10]:
from sklearn.pipeline import Pipeline

In [36]:
from sklearn.linear_model import LogisticRegression

## 文本转稀疏向量矩阵、TF-IDF计算单词重要性

In [4]:
count = CountVectorizer()#词频：训练后生成一个词频字典

In [5]:
tfidf = TfidfTransformer()

In [6]:
np.set_printoptions(precision=2)#设置numpy显示

In [7]:
docs = np.array([
    'The sun is shining',
    'The weather is sweet',
    'The sun is shining and the weather is sweet'
])

In [8]:
bag = count.fit_transform(docs)#生成特征词向量的压缩行稀疏矩阵

In [9]:
print(count.vocabulary_) #返回一个词频字典

{'the': 5, 'sun': 3, 'is': 1, 'shining': 2, 'weather': 6, 'sweet': 4, 'and': 0}


In [17]:
tff = tfidf.fit_transform(bag) #计算词向量矩阵的TFIDF矩阵，并进行归一化处理

In [19]:
bag.toarray()#将压缩行系数矩阵转为ndarray格式

array([[0, 1, 1, 1, 0, 1, 0],
       [0, 1, 0, 0, 1, 1, 1],
       [1, 2, 1, 1, 1, 2, 1]], dtype=int64)

In [22]:
tff.toarray()

array([[0.  , 0.43, 0.56, 0.56, 0.  , 0.43, 0.  ],
       [0.  , 0.43, 0.  , 0.  , 0.56, 0.43, 0.56],
       [0.4 , 0.48, 0.31, 0.31, 0.31, 0.48, 0.31]])

## 清洗文本数据

In [11]:
df = pd.read_csv('/home/fannian/tool_data/outdata/movie_data.csv')

In [31]:
dr = df.loc[0,'review'][-50:]

In [12]:
def preprocessor(text):
    text = re.sub('<[^>]*>','',text)#正则表达式将Html符号替换掉
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',text)#找出表情符号并以列表形式存储
    #替换所有非单词字符,将文本转换为小写，并将表情符号追加到文本之后（删除代表鼻子的字符-）
    text = re.sub('[\W]+',' ',text.lower()) + ''.join(emoticons).replace('-','')
    return text

In [35]:
preprocessor(dr)

'is seven title brazil not available'

In [36]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :):(:)'

In [21]:
df['review'] = df['review'].apply(preprocessor)

## 标记文档

In [13]:
porter = PorterStemmer() #词干提取算法

In [14]:
def tokenizer_porter(text):
    """以空格符分割字符串，返回一个单词列表,并对该列表每个元素进行词干提取"""
    return [porter.stem(word) for word in text.split()]

In [15]:
def tokenizer(text):
    """以空格符分割字符串，返回一个单词列表"""
    return text.split()

In [53]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

In [16]:
stop = stopwords.words('english')

In [54]:
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# 逻辑回归应用于文档分类

In [22]:
X = df.loc[:,'review'].values

In [23]:
y = df.loc[:,'sentiment'].values

In [24]:
X_train, X_test,y_train,y_test = train_test_split(X,y,test_size=0.5)

In [30]:
nb = MultinomialNB(alpha=0.1)

In [37]:
lr = LogisticRegression(C=10.0,penalty='l2')

In [41]:
tfidf_vect =  TfidfVectorizer(stop_words=None,tokenizer=tokenizer)

In [42]:
nb_tfidf = Pipeline([('vect',tfidf_vect),('clf',lr)])

In [43]:
nb_tfidf.fit(X_train,y_train)

/mnt/d/ProgramFiles_linux/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [44]:
nb_tfidf.score(X_test,y_test)

0.89576